In [0]:
 %tensorflow_version 1.x
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.resnet50 import preprocess_input as resnet_preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.applications import VGG16
from keras.models import Sequential, Model
from keras.layers import Dense, Input, Layer, Flatten, Activation, GlobalAveragePooling2D
from keras import callbacks
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import os
from sklearn.utils import class_weight
import math
#from keras.applications import resnet50
from keras.applications import resnet
import matplotlib.pyplot as plt
from keras.applications.resnet50 import ResNet50 

Helper function for printing pretty confusion matrix

In [0]:

import numpy as np
import matplotlib.pyplot as plt
import itertools

from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels

#https://github.com/onur94/keras/blob/master/PlotConfusionMatrix.py

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    #print(cm)
    plt.figure(figsize=(10, 10))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('Actual label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    plt.show()

Set seed to fixed value

In [0]:
# Seed value
# Apparently you may use different seed values at each stage
# https://stackoverflow.com/questions/50659482/why-cant-i-get-reproducible-results-in-keras-even-though-i-set-the-random-seeds
SEED_VALUE= 42

# 1. Set `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['PYTHONHASHSEED']=str(SEED_VALUE)

# 2. Set `python` built-in pseudo-random generator at a fixed value
import random
random.seed(SEED_VALUE)

# 3. Set `numpy` pseudo-random generator at a fixed value
import numpy as np
np.random.seed(SEED_VALUE)

# 4. Set the `tensorflow` pseudo-random generator at a fixed value
import tensorflow as tf
tf.compat.v1.set_random_seed(SEED_VALUE)

# 5. Configure a new global `tensorflow` session
from keras import backend as K
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)
# for later versions:
# session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
# sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
# tf.compat.v1.keras.backend.set_session(sess)

In [1]:
#@title Copy binary data to /content/{train, test, validation}

!cp source /content/

cp: cannot stat 'source': No such file or directory


Setup generators

In [0]:
BATCH_SIZE = 64

datagen = ImageDataGenerator(preprocess_input)
train_generator = datagen.flow_from_directory(
        '/content/train',
        target_size=(224, 224),
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle=True)

validation_generator = datagen.flow_from_directory(
        '/content/validation',
        target_size=(224, 224),
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle=False)

test_generator = datagen.flow_from_directory(
        '/content/test',
        target_size=(224, 224),
        batch_size=BATCH_SIZE,
        class_mode=None,
        shuffle=False)




Setup of early stopping

In [0]:
early_stopping = callbacks.EarlyStopping(monitor='val_loss',
    min_delta=0,
    patience=4,
    verbose=1,
    mode='auto',
    restore_best_weights=True
)

Base binary model training

In [0]:
vgg_model = VGG16(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

x = Flatten()(vgg_model.output)
x = Dense(128, activation='relu')(x)
output = Dense(2, activation='softmax')(x)

model = Model(inputs=vgg_model.inputs, outputs=output)

for layer in model.layers[:-2]:
    layer.trainable = False
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy']
)
model.fit_generator(train_generator,
                    steps_per_epoch=(NUMBER_OF_TRAIN_DATA//BATCH_SIZE)+1,
                    validation_data=validation_generator,
                    validation_steps=(NUMBER_OF_VALIDATION_DATA//BATCH_SIZE)+1,
                    epochs=30,
                    verbose=1,
                    class_weight=class_weights,
                    callbacks=[early_stopping])

In [0]:
Y_pred = model.predict_generator(test_generator, (NUMBER_OF_TEST_DATA // BATCH_SIZE) + 1)
y_pred = np.argmax(Y_pred, axis=1)
cm = confusion_matrix(test_generator.classes, y_pred)
print('Classification Report')
target_names = train_generator.class_indices.keys()
print(classification_report(test_generator.classes, y_pred, target_names=target_names))
plot_confusion_matrix(cm, target_names, normalize=True)

Enhanced binary model

In [0]:
vgg_model = VGG16(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

x = Flatten()(vgg_model.output)
x = Dense(128, activation='relu')(x)
x = Dense(128, activation='relu')(x)
x = Dense(64, activation='relu')(x)
x = Dense(64, activation='relu')(x)
output = Dense(2, activation='softmax')(x)

model = Model(inputs=vgg_model.inputs, outputs=output)

for layer in model.layers[:-7]:
    layer.trainable = False
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy']
)
model.fit_generator(train_generator,
                    steps_per_epoch=(NUMBER_OF_TRAIN_DATA//BATCH_SIZE)+1,
                    validation_data=validation_generator,
                    validation_steps=(NUMBER_OF_VALIDATION_DATA//BATCH_SIZE)+1,
                    epochs=30,
                    verbose=1,
                    class_weight=class_weights,
                    callbacks=[early_stopping])

In [0]:
Y_pred = model.predict_generator(test_generator, (NUMBER_OF_TEST_DATA // BATCH_SIZE) + 1)
y_pred = np.argmax(Y_pred, axis=1)
cm = confusion_matrix(test_generator.classes, y_pred)
print('Classification Report')
target_names = train_generator.class_indices.keys()
print(classification_report(test_generator.classes, y_pred, target_names=target_names))
plot_confusion_matrix(cm, target_names, normalize=True)

ResNet50 model. It has different preprocessing function, so it does need to reinitialize generators

In [0]:
datagen = ImageDataGenerator(resnet_preprocess_input)
train_generator = datagen.flow_from_directory(
        '/content/train',
        target_size=(224, 224),
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle=True)

validation_generator = datagen.flow_from_directory(
        '/content/validation',
        target_size=(224, 224),
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle=False)

test_generator = datagen.flow_from_directory(
        '/content/test',
        target_size=(224, 224),
        batch_size=BATCH_SIZE,
        class_mode=None,
        shuffle=False)


In [0]:
resnet_model = ResNet50(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

x = Flatten()(resnet_model.output)
x = Dense(128, activation='relu')(x)
x = Dense(128, activation='relu')(x)
x = Dense(128, activation='relu')(x)
output = Dense(2, activation='softmax')(x)

model = Model(inputs=resnet_model.inputs, outputs=output)
for layer in model.layers[:-12]:
    layer.trainable = False
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy']
)
model.fit_generator(train_generator,
                    steps_per_epoch=(NUMBER_OF_TRAIN_DATA//BATCH_SIZE)+1,
                    validation_data=validation_generator,
                    validation_steps=(NUMBER_OF_VALIDATION_DATA//BATCH_SIZE)+1,
                    epochs=30,
                    verbose=1,
                    class_weight=class_weights,
                    callbacks=[early_stopping])

In [0]:
Y_pred = model.predict_generator(test_generator, (NUMBER_OF_TEST_DATA // BATCH_SIZE) + 1)
y_pred = np.argmax(Y_pred, axis=1)
cm = confusion_matrix(test_generator.classes, y_pred)
print('Classification Report')
target_names = train_generator.class_indices.keys()
print(classification_report(test_generator.classes, y_pred, target_names=target_names))
plot_confusion_matrix(cm, target_names, normalize=True)

Copy class data into same structure

In [0]:
!rm -rf /content
!cp source /content/{train, test, validation}

In [0]:
BATCH_SIZE = 64

datagen = ImageDataGenerator(preprocess_input)
train_generator = datagen.flow_from_directory(
        '/content/train',
        target_size=(224, 224),
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle=True)

validation_generator = datagen.flow_from_directory(
        '/content/validation',
        target_size=(224, 224),
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle=False)

test_generator = datagen.flow_from_directory(
        '/content/test',
        target_size=(224, 224),
        batch_size=BATCH_SIZE,
        class_mode=None,
        shuffle=False)


Calculate class weight

In [0]:
number_of_examples = len(validation_generator.filenames)
number_of_generator_calls = math.ceil(number_of_examples / (1.0 * BATCH_SIZE)) 

validation_labels = []

for i in range(0,int(number_of_generator_calls)):
    _, labels = next(validation_generator)
    labels = (labels * np.array([np.arange(15) for _ in range(len(labels))])).sum(axis=1)
    for label in labels:
        validation_labels.append(label)

class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(validation_labels),
                                                 validation_labels)

Classification model

In [0]:
vgg_model = VGG16(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

x = Flatten()(vgg_model.output)
x = Dense(128, activation='relu')(x)
output = Dense(16, activation='softmax')(x)

model = Model(inputs=vgg_model.inputs, outputs=output)
for layer in model.layers[:-2]:
    layer.trainable = False
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy']
)
model.fit_generator(train_generator,
                    steps_per_epoch=(NUMBER_OF_TRAIN_DATA//BATCH_SIZE)+1,
                    validation_data=validation_generator,
                    validation_steps=(NUMBER_OF_VALIDATION_DATA//BATCH_SIZE)+1,
                    epochs=30,
                    verbose=1,
                    class_weight=class_weights,
                    callbacks=[early_stopping])


In [0]:
Y_pred = model.predict_generator(test_generator, (NUMBER_OF_TEST_DATA // BATCH_SIZE) + 1)
y_pred = np.argmax(Y_pred, axis=1)
cm = confusion_matrix(test_generator.classes, y_pred)
print('Classification Report')
target_names = train_generator.class_indices.keys()
print(classification_report(test_generator.classes, y_pred, target_names=target_names))
plot_confusion_matrix(cm, target_names, normalize=True)

Model with 1% better accuracy

In [0]:
vgg_model = VGG16(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

x = Flatten()(vgg_model.output)
x = Dense(128, activation='relu')(x)
x = Dense(128, activation='relu')(x)
output = Dense(16, activation='softmax')(x)

model = Model(inputs=vgg_model.inputs, outputs=output)
for layer in model.layers[:-2]:
    layer.trainable = False
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy']
)
model.fit_generator(train_generator,
                    steps_per_epoch=(NUMBER_OF_TRAIN_DATA//BATCH_SIZE)+1,
                    validation_data=validation_generator,
                    validation_steps=(NUMBER_OF_VALIDATION_DATA//BATCH_SIZE)+1,
                    epochs=30,
                    verbose=1,
                    class_weight=class_weights,
                    callbacks=[early_stopping])

In [0]:
Y_pred = model.predict_generator(test_generator, (NUMBER_OF_TEST_DATA // BATCH_SIZE) + 1)
y_pred = np.argmax(Y_pred, axis=1)
cm = confusion_matrix(test_generator.classes, y_pred)
print('Classification Report')
target_names = train_generator.class_indices.keys()
print(classification_report(test_generator.classes, y_pred, target_names=target_names))
plot_confusion_matrix(cm, target_names, normalize=True)

ResNet50

In [0]:
datagen = ImageDataGenerator(resnet_preprocess_input)
train_generator = datagen.flow_from_directory(
        '/content/train',
        target_size=(224, 224),
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle=True)

validation_generator = datagen.flow_from_directory(
        '/content/validation',
        target_size=(224, 224),
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle=False)

test_generator = datagen.flow_from_directory(
        '/content/test',
        target_size=(224, 224),
        batch_size=BATCH_SIZE,
        class_mode=None,
        shuffle=False)


In [0]:
resnet_model = ResNet50(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

x = Flatten()(resnet_model.output)
x = Dense(128, activation='relu')(x)
x = Dense(128, activation='relu')(x)
x = Dense(128, activation='relu')(x)
output = Dense(16, activation='softmax')(x)

model = Model(inputs=resnet_model.inputs, outputs=output)
for layer in model.layers[:-9]:
    layer.trainable = False
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy']
)
model.fit_generator(train_generator,
                    steps_per_epoch=(NUMBER_OF_TRAIN_DATA//BATCH_SIZE)+1,
                    validation_data=validation_generator,
                    validation_steps=(NUMBER_OF_VALIDATION_DATA//BATCH_SIZE)+1,
                    epochs=30,
                    verbose=1,
                    class_weight=class_weights,
                    callbacks=[early_stopping])

In [0]:
Y_pred = model.predict_generator(test_generator, (NUMBER_OF_TEST_DATA // BATCH_SIZE) + 1)
y_pred = np.argmax(Y_pred, axis=1)
cm = confusion_matrix(test_generator.classes, y_pred)
print('Classification Report')
target_names = train_generator.class_indices.keys()
print(classification_report(test_generator.classes, y_pred, target_names=target_names))
plot_confusion_matrix(cm, target_names, normalize=True)

Remove healhy_images class from dataset

In [0]:
!rm -rf content/test/healthy_images content/train/healthy content/validation/healthy_images

In [0]:
BATCH_SIZE = 64

datagen = ImageDataGenerator(preprocess_input)
train_generator = datagen.flow_from_directory(
        '/content/train',
        target_size=(224, 224),
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle=True)

validation_generator = datagen.flow_from_directory(
        '/content/validation',
        target_size=(224, 224),
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle=False)

test_generator = datagen.flow_from_directory(
        '/content/test',
        target_size=(224, 224),
        batch_size=BATCH_SIZE,
        class_mode=None,
        shuffle=False)


In [0]:
number_of_examples = len(validation_generator.filenames)
number_of_generator_calls = math.ceil(number_of_examples / (1.0 * BATCH_SIZE)) 

validation_labels = []

for i in range(0,int(number_of_generator_calls)):
    _, labels = next(validation_generator)
    labels = (labels * np.array([np.arange(15) for _ in range(len(labels))])).sum(axis=1)
    for label in labels:
        validation_labels.append(label)

class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(validation_labels),
                                                 validation_labels)

Model trained only on faulty data

In [0]:
vgg_model = VGG16(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

x = Flatten()(vgg_model.output)
x = Dense(128, activation='relu')(x)
x = Dense(128, activation='relu')(x)
x = Dense(128, activation='relu')(x)
output = Dense(15, activation='softmax')(x)

model = Model(inputs=vgg_model.inputs, outputs=output)
for layer in model.layers[:-12]:
    layer.trainable = False
model.compile(optimizer=optimizers.Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy']
)
model.fit_generator(train_generator,
                    steps_per_epoch=(NUMBER_OF_TRAIN_DATA//BATCH_SIZE)+1,
                    validation_data=validation_generator,
                    validation_steps=(NUMBER_OF_VALIDATION_DATA//BATCH_SIZE)+1,
                    epochs=30,
                    verbose=1,
                    class_weight=class_weights,
                    callbacks=[early_stopping])

In [0]:
Y_pred = model.predict_generator(test_generator, (NUMBER_OF_TEST_DATA // BATCH_SIZE) + 1)
y_pred = np.argmax(Y_pred, axis=1)
cm = confusion_matrix(test_generator.classes, y_pred)
print('Classification Report')
target_names = train_generator.class_indices.keys()
print(classification_report(test_generator.classes, y_pred, target_names=target_names))
plot_confusion_matrix(cm, target_names, normalize=True)